1.Decompose the xml file of ground truth in the training set and write it into the fish_train.txt file.

In [3]:
import xml.dom.minidom
import os
import cv2

def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        L.append(files)
    return L

def file_name1(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.flv':
                L.append(os.path.splitext(file)[0])
    return L
fish_name = ['dascyllus reticulatus','chaetodon lunulatus','pempheris vanicolensis','dascyllus aruanus','plectrogly-phidodon dickii',
             'amphiprion clarkii','chaetodon trifascialis','acanthurus nigrofuscus','chromis chrysura','hemigymnus melapterus',
             'myripristis kuntee','chaetodon speculum','abudefduf vaigiensis','neoglyphidodon nigroris','zebrasoma scopas']
    # The os.path.splitext() function splits the path into file name + extension
file_dir = 'data/fishclef_2015/training_set/videos'
L = file_name1(file_dir)
f = open('data/fish_train.txt','w')
for iter in range(0,len(L)):
    if iter==15:
        print(L[iter])
    # Open xml docunment
    dom = xml.dom.minidom.parse('data/fishclef_2015/training_set/gt' + '/' + L[iter]+'.xml')
    # Get ducunemnt objencts
    root = dom.documentElement
    itemlist = root.getElementsByTagName('frame')
    for item in itemlist:
        id = item.getAttribute('id')
        obect = item.getElementsByTagName('object')
        for i in range(0,len(obect)):

            h= obect[i].getAttribute('h')
            w= obect[i].getAttribute('w')
            x= obect[i].getAttribute('x')
            y= obect[i].getAttribute('y')
            fish_species = obect[i].getAttribute('fish_species').lower()
            if fish_species=="hemigumnus malapterus":
                fish_species = "hemigymnus melapterus"
            if fish_species=="chaetodon lununatus":
                fish_species = "chaetodon lunulatus"

            if fish_species in fish_name:
                f.writelines([str(iter+1) + '_' + id +'.jpg', ' ', fish_species, ' ', x, ' ', y, ' ', str(int(x)+int(w)), ' ', str(int(y)+int(h))+'\n'])
f.close()


9c333821ab0e2a9e4c5209065c415309#201102031130_s3_0


2.Extract all the training set videos into a single frame image and save it in the fish_train folder

In [4]:
import os
import cv2
import shutil

def file_name(file_dir):
    L = []
    for root, dir, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.flv':
                L.append(file)
    return L
  
file_dir = 'data/fishclef_2015/training_set/videos'
L = file_name(file_dir)
print (len(L))
save_images_path = "data/fish_train/"
if os.path.exists(save_images_path):
    shutil.rmtree(save_images_path)
os.makedirs(save_images_path)
for iter in range(0,len(L)):
    video_full_path = "data/fishclef_2015/training_set/videos/" + L[iter]
    cap = cv2.VideoCapture(video_full_path)
    #print cap.isOpened()
    frame_count = 0
    success = True
    while (success):
        success, frame = cap.read()
        if not success:
            break
        #print 'Read a new frame: ', success
        params = []
        # params.append(cv.CV_IMWRITE_PXM_BINARY)
        #print(frame_count)
        params.append(1)
        cv2.imwrite(save_images_path + str(iter + 1) + "_%d.jpg" % frame_count, frame, params)

        frame_count = frame_count + 1

    cap.release()


20


3.Convert the training set to yolo format

In [5]:
import os, sys
import glob
import random
from PIL import Image
import json

# Create yolo txt
def Create_txt(yolo_path, yolo_name, img_path, img_name, all_loc, class_dict,label_list):
    yolo = os.path.join(yolo_path, yolo_name)
    if os.path.exists(yolo):
        os.remove(yolo)
     # write object info into txt
    with open(yolo, "w") as f:
        for index,loc in enumerate(all_loc):
            img = Image.open(os.path.join(img_path, img_name))
            [img_width, img_height] = img.size
        
            # Further check the data, some labeling information may have w or h being 0, 
            # such data will lead to the calculation of regression loss as nan
            xmin = min(max(0, all_loc[index][0]), img_width)
            ymin = min(max(0, all_loc[index][1]), img_height)
            xmax = min(max(0, all_loc[index][2]), img_width)
            ymax = min(max(0, all_loc[index][3]), img_height)
            if xmax <= xmin or ymax <= ymin:
                print("Warning: in '{}' xml, there are some bbox w/h <=0")
                continue
            # Convert box information to yolo format
            xcenter = xmin + (xmax - xmin) / 2
            ycenter = ymin + (ymax - ymin) / 2
            w = xmax - xmin
            h = ymax - ymin

            # Convert absolute coordinates to relative coordinates, save 6 decimal places
            xcenter = round(xcenter / img_width, 6)
            ycenter = round(ycenter / img_height, 6)
            w = round(w / img_width, 6)
            h = round(h / img_height, 6)
            class_index = class_dict[label_list[index]]
            info = [str(i) for i in [class_index, xcenter, ycenter, w, h]]
            if index == 0:
                f.write(" ".join(info))
            else:
                f.write("\n" + " ".join(info))
file = 'data/fish_train.txt'
img_dir = "data//fish_train"
txt_dir = 'data/train_txt'
label_json_path = 'data/fish_classes.json'
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

img_num = 0
line_num = 0
img_list = []
# read class_indict
json_file = open(label_json_path, 'r')
class_dict = json.load(json_file)
with open(file, 'r') as fin:
    lines = fin.readlines()
    for idx in range(len(lines)):
        label_list = []
        all_loc = []
        line_num += 1

        img_name, label, label1, xmin, ymin, xmax, ymax = lines[idx].strip().split(' ')
        #img_name='test_' + img_name
        if img_name not in img_list:
            img_list.append(img_name)
            img_num += 1
            
            label_list.append(label + ' ' + label1)
            all_loc.append([int(xmin), int(ymin), int(xmax), int(ymax)])

            for idx2 in range(idx+1, len(lines)):
                img_name2, label2, label3, xmin2, ymin2, xmax2, ymax2 = lines[idx2].strip().split(' ')
                if img_name == img_name2:
                    label_list.append(label2 + ' ' + label3)
                    all_loc.append([int(xmin2), int(ymin2), int(xmax2), int(ymax2)])

            anno_name = str(img_name.split('.')[0]) + '.txt'
            Create_txt(txt_dir, anno_name, img_dir, img_name, all_loc,class_dict,label_list)

        print ('line_num: ' + str(line_num) + '; ' + 'img_num: ' + str(img_num) + '; '  + 'box_num: ' + str(len(all_loc)) + '; ' + img_name)

line_num: 1; img_num: 1; box_num: 1; 1_0.jpg
line_num: 2; img_num: 2; box_num: 1; 1_1.jpg
line_num: 3; img_num: 3; box_num: 1; 1_2.jpg
line_num: 4; img_num: 4; box_num: 2; 1_3.jpg
line_num: 5; img_num: 4; box_num: 0; 1_3.jpg
line_num: 6; img_num: 5; box_num: 3; 1_4.jpg
line_num: 7; img_num: 5; box_num: 0; 1_4.jpg
line_num: 8; img_num: 5; box_num: 0; 1_4.jpg
line_num: 9; img_num: 6; box_num: 3; 1_5.jpg
line_num: 10; img_num: 6; box_num: 0; 1_5.jpg
line_num: 11; img_num: 6; box_num: 0; 1_5.jpg
line_num: 12; img_num: 7; box_num: 2; 1_6.jpg
line_num: 13; img_num: 7; box_num: 0; 1_6.jpg
line_num: 14; img_num: 8; box_num: 2; 1_7.jpg
line_num: 15; img_num: 8; box_num: 0; 1_7.jpg
line_num: 16; img_num: 9; box_num: 2; 1_8.jpg
line_num: 17; img_num: 9; box_num: 0; 1_8.jpg
line_num: 18; img_num: 10; box_num: 3; 1_9.jpg
line_num: 19; img_num: 10; box_num: 0; 1_9.jpg
line_num: 20; img_num: 10; box_num: 0; 1_9.jpg
line_num: 21; img_num: 11; box_num: 2; 1_10.jpg
line_num: 22; img_num: 11; box_num: 0;

line_num: 173; img_num: 61; box_num: 2; 1_60.jpg
line_num: 174; img_num: 61; box_num: 0; 1_60.jpg
line_num: 175; img_num: 62; box_num: 2; 1_61.jpg
line_num: 176; img_num: 62; box_num: 0; 1_61.jpg
line_num: 177; img_num: 63; box_num: 2; 1_62.jpg
line_num: 178; img_num: 63; box_num: 0; 1_62.jpg
line_num: 179; img_num: 64; box_num: 3; 1_63.jpg
line_num: 180; img_num: 64; box_num: 0; 1_63.jpg
line_num: 181; img_num: 64; box_num: 0; 1_63.jpg
line_num: 182; img_num: 65; box_num: 3; 1_64.jpg
line_num: 183; img_num: 65; box_num: 0; 1_64.jpg
line_num: 184; img_num: 65; box_num: 0; 1_64.jpg
line_num: 185; img_num: 66; box_num: 3; 1_65.jpg
line_num: 186; img_num: 66; box_num: 0; 1_65.jpg
line_num: 187; img_num: 66; box_num: 0; 1_65.jpg
line_num: 188; img_num: 67; box_num: 3; 1_66.jpg
line_num: 189; img_num: 67; box_num: 0; 1_66.jpg
line_num: 190; img_num: 67; box_num: 0; 1_66.jpg
line_num: 191; img_num: 68; box_num: 2; 1_67.jpg
line_num: 192; img_num: 68; box_num: 0; 1_67.jpg
line_num: 193; img_n

line_num: 359; img_num: 120; box_num: 4; 1_119.jpg
line_num: 360; img_num: 120; box_num: 0; 1_119.jpg
line_num: 361; img_num: 120; box_num: 0; 1_119.jpg
line_num: 362; img_num: 120; box_num: 0; 1_119.jpg
line_num: 363; img_num: 121; box_num: 5; 1_120.jpg
line_num: 364; img_num: 121; box_num: 0; 1_120.jpg
line_num: 365; img_num: 121; box_num: 0; 1_120.jpg
line_num: 366; img_num: 121; box_num: 0; 1_120.jpg
line_num: 367; img_num: 121; box_num: 0; 1_120.jpg
line_num: 368; img_num: 122; box_num: 5; 1_121.jpg
line_num: 369; img_num: 122; box_num: 0; 1_121.jpg
line_num: 370; img_num: 122; box_num: 0; 1_121.jpg
line_num: 371; img_num: 122; box_num: 0; 1_121.jpg
line_num: 372; img_num: 122; box_num: 0; 1_121.jpg
line_num: 373; img_num: 123; box_num: 5; 1_122.jpg
line_num: 374; img_num: 123; box_num: 0; 1_122.jpg
line_num: 375; img_num: 123; box_num: 0; 1_122.jpg
line_num: 376; img_num: 123; box_num: 0; 1_122.jpg
line_num: 377; img_num: 123; box_num: 0; 1_122.jpg
line_num: 378; img_num: 124; bo

line_num: 530; img_num: 209; box_num: 2; 1_227.jpg
line_num: 531; img_num: 209; box_num: 0; 1_227.jpg
line_num: 532; img_num: 210; box_num: 3; 1_228.jpg
line_num: 533; img_num: 210; box_num: 0; 1_228.jpg
line_num: 534; img_num: 210; box_num: 0; 1_228.jpg
line_num: 535; img_num: 211; box_num: 3; 1_229.jpg
line_num: 536; img_num: 211; box_num: 0; 1_229.jpg
line_num: 537; img_num: 211; box_num: 0; 1_229.jpg
line_num: 538; img_num: 212; box_num: 4; 1_230.jpg
line_num: 539; img_num: 212; box_num: 0; 1_230.jpg
line_num: 540; img_num: 212; box_num: 0; 1_230.jpg
line_num: 541; img_num: 212; box_num: 0; 1_230.jpg
line_num: 542; img_num: 213; box_num: 4; 1_231.jpg
line_num: 543; img_num: 213; box_num: 0; 1_231.jpg
line_num: 544; img_num: 213; box_num: 0; 1_231.jpg
line_num: 545; img_num: 213; box_num: 0; 1_231.jpg
line_num: 546; img_num: 214; box_num: 4; 1_232.jpg
line_num: 547; img_num: 214; box_num: 0; 1_232.jpg
line_num: 548; img_num: 214; box_num: 0; 1_232.jpg
line_num: 549; img_num: 214; bo

line_num: 759; img_num: 292; box_num: 1; 2_32.jpg
line_num: 760; img_num: 293; box_num: 1; 2_33.jpg
line_num: 761; img_num: 294; box_num: 1; 2_34.jpg
line_num: 762; img_num: 295; box_num: 2; 2_35.jpg
line_num: 763; img_num: 295; box_num: 0; 2_35.jpg
line_num: 764; img_num: 296; box_num: 2; 2_36.jpg
line_num: 765; img_num: 296; box_num: 0; 2_36.jpg
line_num: 766; img_num: 297; box_num: 2; 2_37.jpg
line_num: 767; img_num: 297; box_num: 0; 2_37.jpg
line_num: 768; img_num: 298; box_num: 2; 2_38.jpg
line_num: 769; img_num: 298; box_num: 0; 2_38.jpg
line_num: 770; img_num: 299; box_num: 2; 2_39.jpg
line_num: 771; img_num: 299; box_num: 0; 2_39.jpg
line_num: 772; img_num: 300; box_num: 1; 2_40.jpg
line_num: 773; img_num: 301; box_num: 1; 2_41.jpg
line_num: 774; img_num: 302; box_num: 1; 2_42.jpg
line_num: 775; img_num: 303; box_num: 1; 2_43.jpg
line_num: 776; img_num: 304; box_num: 1; 2_44.jpg
line_num: 777; img_num: 305; box_num: 1; 2_45.jpg
line_num: 778; img_num: 306; box_num: 1; 2_46.jpg


...

line_num: 8334; img_num: 4688; box_num: 3; 18_852.jpg
line_num: 8335; img_num: 4688; box_num: 0; 18_852.jpg
line_num: 8336; img_num: 4688; box_num: 0; 18_852.jpg
line_num: 8337; img_num: 4689; box_num: 3; 18_853.jpg
line_num: 8338; img_num: 4689; box_num: 0; 18_853.jpg
line_num: 8339; img_num: 4689; box_num: 0; 18_853.jpg
line_num: 8340; img_num: 4690; box_num: 3; 18_854.jpg
line_num: 8341; img_num: 4690; box_num: 0; 18_854.jpg
line_num: 8342; img_num: 4690; box_num: 0; 18_854.jpg
line_num: 8343; img_num: 4691; box_num: 3; 18_855.jpg
line_num: 8344; img_num: 4691; box_num: 0; 18_855.jpg
line_num: 8345; img_num: 4691; box_num: 0; 18_855.jpg
line_num: 8346; img_num: 4692; box_num: 3; 18_856.jpg
line_num: 8347; img_num: 4692; box_num: 0; 18_856.jpg
line_num: 8348; img_num: 4692; box_num: 0; 18_856.jpg
line_num: 8349; img_num: 4693; box_num: 3; 18_857.jpg
line_num: 8350; img_num: 4693; box_num: 0; 18_857.jpg
line_num: 8351; img_num: 4693; box_num: 0; 18_857.jpg
line_num: 8352; img_num: 469

line_num: 8544; img_num: 4776; box_num: 3; 18_940.jpg
line_num: 8545; img_num: 4776; box_num: 0; 18_940.jpg
line_num: 8546; img_num: 4776; box_num: 0; 18_940.jpg
line_num: 8547; img_num: 4777; box_num: 3; 18_941.jpg
line_num: 8548; img_num: 4777; box_num: 0; 18_941.jpg
line_num: 8549; img_num: 4777; box_num: 0; 18_941.jpg
line_num: 8550; img_num: 4778; box_num: 3; 18_942.jpg
line_num: 8551; img_num: 4778; box_num: 0; 18_942.jpg
line_num: 8552; img_num: 4778; box_num: 0; 18_942.jpg
line_num: 8553; img_num: 4779; box_num: 3; 18_943.jpg
line_num: 8554; img_num: 4779; box_num: 0; 18_943.jpg
line_num: 8555; img_num: 4779; box_num: 0; 18_943.jpg
line_num: 8556; img_num: 4780; box_num: 3; 18_944.jpg
line_num: 8557; img_num: 4780; box_num: 0; 18_944.jpg
line_num: 8558; img_num: 4780; box_num: 0; 18_944.jpg
line_num: 8559; img_num: 4781; box_num: 1; 18_945.jpg
line_num: 8560; img_num: 4782; box_num: 1; 18_946.jpg
line_num: 8561; img_num: 4783; box_num: 1; 18_947.jpg
line_num: 8562; img_num: 478

line_num: 8707; img_num: 4900; box_num: 1; 19_207.jpg
line_num: 8708; img_num: 4901; box_num: 1; 19_208.jpg
line_num: 8709; img_num: 4902; box_num: 1; 19_209.jpg
line_num: 8710; img_num: 4903; box_num: 1; 19_210.jpg
line_num: 8711; img_num: 4904; box_num: 1; 19_211.jpg
line_num: 8712; img_num: 4905; box_num: 1; 19_212.jpg
line_num: 8713; img_num: 4906; box_num: 1; 19_213.jpg
line_num: 8714; img_num: 4907; box_num: 1; 19_214.jpg
line_num: 8715; img_num: 4908; box_num: 1; 19_215.jpg
line_num: 8716; img_num: 4909; box_num: 1; 19_216.jpg
line_num: 8717; img_num: 4910; box_num: 1; 19_217.jpg
line_num: 8718; img_num: 4911; box_num: 1; 19_218.jpg
line_num: 8719; img_num: 4912; box_num: 1; 19_219.jpg
line_num: 8720; img_num: 4913; box_num: 1; 19_220.jpg
line_num: 8721; img_num: 4914; box_num: 1; 19_221.jpg
line_num: 8722; img_num: 4915; box_num: 1; 19_222.jpg
line_num: 8723; img_num: 4916; box_num: 1; 19_223.jpg
line_num: 8724; img_num: 4917; box_num: 1; 19_224.jpg
line_num: 8725; img_num: 491

line_num: 8861; img_num: 5044; box_num: 2; 19_378.jpg
line_num: 8862; img_num: 5044; box_num: 0; 19_378.jpg
line_num: 8863; img_num: 5045; box_num: 2; 19_379.jpg
line_num: 8864; img_num: 5045; box_num: 0; 19_379.jpg
line_num: 8865; img_num: 5046; box_num: 1; 19_382.jpg
line_num: 8866; img_num: 5047; box_num: 1; 19_383.jpg
line_num: 8867; img_num: 5048; box_num: 2; 19_384.jpg
line_num: 8868; img_num: 5048; box_num: 0; 19_384.jpg
line_num: 8869; img_num: 5049; box_num: 2; 19_385.jpg
line_num: 8870; img_num: 5049; box_num: 0; 19_385.jpg
line_num: 8871; img_num: 5050; box_num: 2; 19_386.jpg
line_num: 8872; img_num: 5050; box_num: 0; 19_386.jpg
line_num: 8873; img_num: 5051; box_num: 2; 19_387.jpg
line_num: 8874; img_num: 5051; box_num: 0; 19_387.jpg
line_num: 8875; img_num: 5052; box_num: 2; 19_388.jpg
line_num: 8876; img_num: 5052; box_num: 0; 19_388.jpg
line_num: 8877; img_num: 5053; box_num: 1; 19_389.jpg
line_num: 8878; img_num: 5054; box_num: 1; 19_390.jpg
line_num: 8879; img_num: 505

line_num: 9157; img_num: 5275; box_num: 1; 20_283.jpg
line_num: 9158; img_num: 5276; box_num: 1; 20_284.jpg
line_num: 9159; img_num: 5277; box_num: 1; 20_285.jpg
line_num: 9160; img_num: 5278; box_num: 1; 20_286.jpg


4.Decompose the gt xml file in the test set and write it into the fish_test_all.txt file.

In [6]:
# -*- coding=utf-8 -*-
import xml.dom.minidom
import os
import cv2

def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        L.append(files)
    return L

def file_name1(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.flv':
                L.append(os.path.splitext(file)[0])
    return L

fish_name = ['dascyllus reticulatus','chaetodon lunulatus','pempheris vanicolensis','dascyllus aruanus','plectrogly-phidodon dickii',
             'amphiprion clarkii','chaetodon trifascialis','acanthurus nigrofuscus','chromis chrysura','hemigymnus melapterus',
             'myripristis kuntee','chaetodon speculum','abudefduf vaigiensis','neoglyphidodon nigroris','zebrasoma scopas']


file_dir = 'data/fishclef_2015/test_set/videos'
L = file_name1(file_dir)
f = open('data/fish_test_all.txt','w')
for iter in range(0,len(L)):

    dom = xml.dom.minidom.parse('data/fishclef_2015/test_set/gt' + '/' + L[iter]+'.xml')

    root = dom.documentElement
    itemlist = root.getElementsByTagName('frame')
    for item in itemlist:
        id = item.getAttribute('id')
        obect = item.getElementsByTagName('object')
        for i in range(0,len(obect)):

            h= obect[i].getAttribute('h')
            w= obect[i].getAttribute('w')
            x= obect[i].getAttribute('x')
            y= obect[i].getAttribute('y')
            fish_species = obect[i].getAttribute('species_name').lower()
            if fish_species=="hemigumnus malapterus":
                fish_species = "hemigymnus melapterus"
            if fish_species=="chaetodon lununatus":
                fish_species = "chaetodon lunulatus"

            if fish_species in fish_name:
                f.writelines(['\n' + str(iter+21) + '_' + id +'.jpg', ' ', fish_species, ' ', x, ' ', y, ' ', str(int(x)+int(w)), ' ', str(int(y)+int(h))])
f.close()


5. Extract all the training set videos into a single frame image at one time and save it in the fish_val folder,extract part of the data from the test level as a verification set

In [7]:
import os
import cv2

def file_name(file_dir):
    L = []
    for root, dir, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.flv':
                L.append(file)
    return L
file_dir = 'data/fishclef_2015/test_set/videos'
L = file_name(file_dir)
print (len(L))
save_images_path = "data/fish_test/"
if os.path.exists(save_images_path):
    shutil.rmtree(save_images_path)
os.makedirs(save_images_path)
for iter in range(0,len(L)):
    video_full_path = "data/fishclef_2015/test_set/videos/" + L[iter]
    cap = cv2.VideoCapture(video_full_path)
    frame_count = 0
    success = True
    while (success):
        success, frame = cap.read()
        if not success:
            break
        params = []
        params.append(1)
        cv2.imwrite(save_images_path+ str(iter + 21) + "_%d.jpg" % frame_count, frame, params)

        frame_count = frame_count + 1

    cap.release()

73


6. Extract 2000 from the test set as the verification set

In [8]:
import random

all_data = "data/fish_test_all.txt"
fish_test = "data/fish_test.txt"
fish_val = "data/fish_val.txt"
with open(all_data,"r") as f_in,open(fish_test,'w') as f_out1, open(fish_val,'w') as f_out2:
    allData = f_in.readlines()
    nums = random.sample(range(0,len(allData)), 2000)
    for idx in range(len(allData)):
        line = allData[idx].strip()
        if not line:
            continue
        if idx in nums:
            f_out2.write(line+"\n")
        else:
            f_out1.write(line+"\n")
    

7、将测试数据转成YOLO格式

In [9]:
import os, sys
import glob
import random
from PIL import Image
import json

# Create yolo txt
def Create_txt(yolo_path, yolo_name, img_path, img_name, all_loc, class_dict,label_list):
    yolo = os.path.join(yolo_path, yolo_name)
    if os.path.exists(yolo):
        os.remove(yolo)
     # write object info into txt
    with open(yolo, "w") as f:
        for index,loc in enumerate(all_loc):
            img = Image.open(os.path.join(img_path, img_name))
            [img_width, img_height] = img.size
      
            xmin = min(max(0, all_loc[index][0]), img_width)
            ymin = min(max(0, all_loc[index][1]), img_height)
            xmax = min(max(0, all_loc[index][2]), img_width)
            ymax = min(max(0, all_loc[index][3]), img_height)
            if xmax <= xmin or ymax <= ymin:
                print("Warning: in '{}' xml, there are some bbox w/h <=0")
                continue
        
            xcenter = xmin + (xmax - xmin) / 2
            ycenter = ymin + (ymax - ymin) / 2
            w = xmax - xmin
            h = ymax - ymin

         
            xcenter = round(xcenter / img_width, 6)
            ycenter = round(ycenter / img_height, 6)
            w = round(w / img_width, 6)
            h = round(h / img_height, 6)
            class_index = class_dict[label_list[index]]
            info = [str(i) for i in [class_index, xcenter, ycenter, w, h]]
            if index == 0:
                f.write(" ".join(info))
            else:
                f.write("\n" + " ".join(info))
file = 'data/fish_test.txt'
img_dir = "data//fish_test"
txt_dir = 'data/test_txt'
label_json_path = 'data/fish_classes.json'
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

img_num = 0
line_num = 0
img_list = []
# read class_indict
json_file = open(label_json_path, 'r')
class_dict = json.load(json_file)
with open(file, 'r') as fin:
    lines = fin.readlines()
    for idx in range(len(lines)):
        label_list = []
        all_loc = []
        line_num += 1

        img_name, label, label1, xmin, ymin, xmax, ymax = lines[idx].strip().split(' ')
        #img_name='test_' + img_name
        if img_name not in img_list:
            img_list.append(img_name)
            img_num += 1
            
            label_list.append(label + ' ' + label1)
            all_loc.append([int(xmin), int(ymin), int(xmax), int(ymax)])

            for idx2 in range(idx+1, len(lines)):
                img_name2, label2, label3, xmin2, ymin2, xmax2, ymax2 = lines[idx2].strip().split(' ')
                if img_name == img_name2:
                    label_list.append(label2 + ' ' + label3)
                    all_loc.append([int(xmin2), int(ymin2), int(xmax2), int(ymax2)])

            anno_name = str(img_name.split('.')[0]) + '.txt'
            Create_txt(txt_dir, anno_name, img_dir, img_name, all_loc,class_dict,label_list)

        print ('line_num: ' + str(line_num) + '; ' + 'img_num: ' + str(img_num) + '; '  + 'box_num: ' + str(len(all_loc)) + '; ' + img_name)

line_num: 1; img_num: 1; box_num: 1; 21_0.jpg
line_num: 2; img_num: 2; box_num: 1; 21_1.jpg
line_num: 3; img_num: 3; box_num: 2; 21_2.jpg
line_num: 4; img_num: 3; box_num: 0; 21_2.jpg
line_num: 5; img_num: 4; box_num: 2; 21_3.jpg
line_num: 6; img_num: 4; box_num: 0; 21_3.jpg
line_num: 7; img_num: 5; box_num: 2; 21_4.jpg
line_num: 8; img_num: 5; box_num: 0; 21_4.jpg
line_num: 9; img_num: 6; box_num: 2; 21_5.jpg
line_num: 10; img_num: 6; box_num: 0; 21_5.jpg
line_num: 11; img_num: 7; box_num: 2; 21_6.jpg
line_num: 12; img_num: 7; box_num: 0; 21_6.jpg
line_num: 13; img_num: 8; box_num: 2; 21_7.jpg
line_num: 14; img_num: 8; box_num: 0; 21_7.jpg
line_num: 15; img_num: 9; box_num: 2; 21_8.jpg
line_num: 16; img_num: 9; box_num: 0; 21_8.jpg
line_num: 17; img_num: 10; box_num: 2; 21_9.jpg
line_num: 18; img_num: 10; box_num: 0; 21_9.jpg
line_num: 19; img_num: 11; box_num: 2; 21_10.jpg
line_num: 20; img_num: 11; box_num: 0; 21_10.jpg
line_num: 21; img_num: 12; box_num: 2; 21_11.jpg
line_num: 22; 

line_num: 188; img_num: 99; box_num: 2; 21_100.jpg
line_num: 189; img_num: 99; box_num: 0; 21_100.jpg
line_num: 190; img_num: 100; box_num: 1; 21_101.jpg
line_num: 191; img_num: 101; box_num: 2; 21_102.jpg
line_num: 192; img_num: 101; box_num: 0; 21_102.jpg
line_num: 193; img_num: 102; box_num: 2; 21_103.jpg
line_num: 194; img_num: 102; box_num: 0; 21_103.jpg
line_num: 195; img_num: 103; box_num: 2; 21_104.jpg
line_num: 196; img_num: 103; box_num: 0; 21_104.jpg
line_num: 197; img_num: 104; box_num: 3; 21_105.jpg
line_num: 198; img_num: 104; box_num: 0; 21_105.jpg
line_num: 199; img_num: 104; box_num: 0; 21_105.jpg
line_num: 200; img_num: 105; box_num: 2; 21_106.jpg
line_num: 201; img_num: 105; box_num: 0; 21_106.jpg
line_num: 202; img_num: 106; box_num: 2; 21_108.jpg
line_num: 203; img_num: 106; box_num: 0; 21_108.jpg
line_num: 204; img_num: 107; box_num: 2; 21_109.jpg
line_num: 205; img_num: 107; box_num: 0; 21_109.jpg
line_num: 206; img_num: 108; box_num: 2; 21_110.jpg
line_num: 207;

line_num: 358; img_num: 231; box_num: 1; 21_281.jpg
line_num: 359; img_num: 232; box_num: 1; 21_282.jpg
line_num: 360; img_num: 233; box_num: 1; 21_283.jpg
line_num: 361; img_num: 234; box_num: 1; 21_284.jpg
line_num: 362; img_num: 235; box_num: 1; 21_285.jpg
line_num: 363; img_num: 236; box_num: 1; 21_286.jpg
line_num: 364; img_num: 237; box_num: 1; 21_287.jpg
line_num: 365; img_num: 238; box_num: 2; 21_288.jpg
line_num: 366; img_num: 238; box_num: 0; 21_288.jpg
line_num: 367; img_num: 239; box_num: 2; 21_289.jpg
line_num: 368; img_num: 239; box_num: 0; 21_289.jpg
line_num: 369; img_num: 240; box_num: 2; 21_290.jpg
line_num: 370; img_num: 240; box_num: 0; 21_290.jpg
line_num: 371; img_num: 241; box_num: 2; 21_291.jpg
line_num: 372; img_num: 241; box_num: 0; 21_291.jpg
line_num: 373; img_num: 242; box_num: 1; 21_292.jpg
line_num: 374; img_num: 243; box_num: 1; 21_294.jpg
line_num: 375; img_num: 244; box_num: 1; 21_295.jpg
line_num: 376; img_num: 245; box_num: 1; 21_296.jpg
line_num: 37

line_num: 534; img_num: 394; box_num: 1; 24_192.jpg
line_num: 535; img_num: 395; box_num: 1; 24_256.jpg
line_num: 536; img_num: 396; box_num: 1; 24_257.jpg
line_num: 537; img_num: 397; box_num: 1; 24_258.jpg
line_num: 538; img_num: 398; box_num: 1; 24_259.jpg
line_num: 539; img_num: 399; box_num: 1; 24_260.jpg
line_num: 540; img_num: 400; box_num: 1; 24_261.jpg
line_num: 541; img_num: 401; box_num: 1; 24_262.jpg
line_num: 542; img_num: 402; box_num: 1; 24_263.jpg
line_num: 543; img_num: 403; box_num: 1; 24_265.jpg
line_num: 544; img_num: 404; box_num: 2; 24_266.jpg
line_num: 545; img_num: 404; box_num: 0; 24_266.jpg
line_num: 546; img_num: 405; box_num: 1; 24_267.jpg
line_num: 547; img_num: 406; box_num: 1; 24_268.jpg
line_num: 548; img_num: 407; box_num: 2; 24_269.jpg
line_num: 549; img_num: 407; box_num: 0; 24_269.jpg
line_num: 550; img_num: 408; box_num: 2; 24_270.jpg
line_num: 551; img_num: 408; box_num: 0; 24_270.jpg
line_num: 552; img_num: 409; box_num: 2; 24_271.jpg
line_num: 55

line_num: 694; img_num: 527; box_num: 1; 25_166.jpg
line_num: 695; img_num: 528; box_num: 1; 25_167.jpg
line_num: 696; img_num: 529; box_num: 1; 25_168.jpg
line_num: 697; img_num: 530; box_num: 1; 25_169.jpg
line_num: 698; img_num: 531; box_num: 1; 25_170.jpg
line_num: 699; img_num: 532; box_num: 1; 25_171.jpg
line_num: 700; img_num: 533; box_num: 1; 25_172.jpg
line_num: 701; img_num: 534; box_num: 1; 25_173.jpg
line_num: 702; img_num: 535; box_num: 1; 25_174.jpg
line_num: 703; img_num: 536; box_num: 1; 25_175.jpg
line_num: 704; img_num: 537; box_num: 1; 25_176.jpg
line_num: 705; img_num: 538; box_num: 1; 25_177.jpg
line_num: 706; img_num: 539; box_num: 1; 25_178.jpg
line_num: 707; img_num: 540; box_num: 1; 25_180.jpg
line_num: 708; img_num: 541; box_num: 1; 25_181.jpg
line_num: 709; img_num: 542; box_num: 1; 25_182.jpg
line_num: 710; img_num: 543; box_num: 1; 25_183.jpg
line_num: 711; img_num: 544; box_num: 1; 25_184.jpg
line_num: 712; img_num: 545; box_num: 1; 25_185.jpg
line_num: 71

...

line_num: 10804; img_num: 7685; box_num: 1; 85_178.jpg
line_num: 10805; img_num: 7686; box_num: 1; 85_180.jpg
line_num: 10806; img_num: 7687; box_num: 1; 85_182.jpg
line_num: 10807; img_num: 7688; box_num: 1; 85_184.jpg
line_num: 10808; img_num: 7689; box_num: 1; 85_185.jpg
line_num: 10809; img_num: 7690; box_num: 1; 85_186.jpg
line_num: 10810; img_num: 7691; box_num: 1; 85_187.jpg
line_num: 10811; img_num: 7692; box_num: 1; 85_189.jpg
line_num: 10812; img_num: 7693; box_num: 1; 85_190.jpg
line_num: 10813; img_num: 7694; box_num: 1; 85_191.jpg
line_num: 10814; img_num: 7695; box_num: 1; 85_192.jpg
line_num: 10815; img_num: 7696; box_num: 1; 85_193.jpg
line_num: 10816; img_num: 7697; box_num: 1; 85_194.jpg
line_num: 10817; img_num: 7698; box_num: 1; 85_195.jpg
line_num: 10818; img_num: 7699; box_num: 1; 85_196.jpg
line_num: 10819; img_num: 7700; box_num: 1; 85_197.jpg
line_num: 10820; img_num: 7701; box_num: 1; 85_198.jpg
line_num: 10821; img_num: 7702; box_num: 1; 85_199.jpg
line_num: 

line_num: 10955; img_num: 7823; box_num: 1; 86_136.jpg
line_num: 10956; img_num: 7824; box_num: 1; 86_137.jpg
line_num: 10957; img_num: 7825; box_num: 1; 86_138.jpg
line_num: 10958; img_num: 7826; box_num: 1; 86_139.jpg
line_num: 10959; img_num: 7827; box_num: 1; 86_140.jpg
line_num: 10960; img_num: 7828; box_num: 1; 86_143.jpg
line_num: 10961; img_num: 7829; box_num: 1; 86_144.jpg
line_num: 10962; img_num: 7830; box_num: 1; 86_145.jpg
line_num: 10963; img_num: 7831; box_num: 1; 86_146.jpg
line_num: 10964; img_num: 7832; box_num: 1; 86_147.jpg
line_num: 10965; img_num: 7833; box_num: 1; 86_148.jpg
line_num: 10966; img_num: 7834; box_num: 1; 86_149.jpg
line_num: 10967; img_num: 7835; box_num: 1; 86_150.jpg
line_num: 10968; img_num: 7836; box_num: 1; 86_151.jpg
line_num: 10969; img_num: 7837; box_num: 1; 86_153.jpg
line_num: 10970; img_num: 7838; box_num: 1; 86_154.jpg
line_num: 10971; img_num: 7839; box_num: 1; 86_155.jpg
line_num: 10972; img_num: 7840; box_num: 1; 86_156.jpg
line_num: 

line_num: 11112; img_num: 7980; box_num: 1; 88_288.jpg
line_num: 11113; img_num: 7981; box_num: 1; 88_289.jpg
line_num: 11114; img_num: 7982; box_num: 1; 88_292.jpg
line_num: 11115; img_num: 7983; box_num: 1; 88_293.jpg
line_num: 11116; img_num: 7984; box_num: 1; 88_294.jpg
line_num: 11117; img_num: 7985; box_num: 1; 88_295.jpg
line_num: 11118; img_num: 7986; box_num: 1; 88_296.jpg
line_num: 11119; img_num: 7987; box_num: 1; 88_297.jpg
line_num: 11120; img_num: 7988; box_num: 1; 88_298.jpg
line_num: 11121; img_num: 7989; box_num: 1; 89_20.jpg
line_num: 11122; img_num: 7990; box_num: 1; 89_21.jpg
line_num: 11123; img_num: 7991; box_num: 1; 89_23.jpg
line_num: 11124; img_num: 7992; box_num: 1; 89_24.jpg
line_num: 11125; img_num: 7993; box_num: 1; 89_25.jpg
line_num: 11126; img_num: 7994; box_num: 1; 90_1.jpg
line_num: 11127; img_num: 7995; box_num: 1; 90_2.jpg
line_num: 11128; img_num: 7996; box_num: 1; 90_3.jpg
line_num: 11129; img_num: 7997; box_num: 1; 90_13.jpg
line_num: 11130; img_n

line_num: 11333; img_num: 8137; box_num: 1; 91_107.jpg
line_num: 11334; img_num: 8138; box_num: 1; 91_108.jpg
line_num: 11335; img_num: 8139; box_num: 1; 91_110.jpg
line_num: 11336; img_num: 8140; box_num: 1; 91_111.jpg
line_num: 11337; img_num: 8141; box_num: 1; 91_112.jpg
line_num: 11338; img_num: 8142; box_num: 1; 91_114.jpg
line_num: 11339; img_num: 8143; box_num: 1; 91_115.jpg
line_num: 11340; img_num: 8144; box_num: 1; 91_150.jpg
line_num: 11341; img_num: 8145; box_num: 1; 91_152.jpg
line_num: 11342; img_num: 8146; box_num: 1; 91_153.jpg
line_num: 11343; img_num: 8147; box_num: 1; 91_154.jpg
line_num: 11344; img_num: 8148; box_num: 1; 91_155.jpg
line_num: 11345; img_num: 8149; box_num: 1; 91_157.jpg
line_num: 11346; img_num: 8150; box_num: 1; 91_159.jpg
line_num: 11347; img_num: 8151; box_num: 1; 91_160.jpg
line_num: 11348; img_num: 8152; box_num: 1; 91_161.jpg
line_num: 11349; img_num: 8153; box_num: 1; 91_162.jpg
line_num: 11350; img_num: 8154; box_num: 1; 91_163.jpg
line_num: 

line_num: 11566; img_num: 8355; box_num: 1; 92_272.jpg
line_num: 11567; img_num: 8356; box_num: 1; 93_1.jpg
line_num: 11568; img_num: 8357; box_num: 1; 93_2.jpg
line_num: 11569; img_num: 8358; box_num: 1; 93_3.jpg
line_num: 11570; img_num: 8359; box_num: 1; 93_4.jpg
line_num: 11571; img_num: 8360; box_num: 1; 93_6.jpg
line_num: 11572; img_num: 8361; box_num: 1; 93_7.jpg
line_num: 11573; img_num: 8362; box_num: 1; 93_8.jpg
line_num: 11574; img_num: 8363; box_num: 1; 93_9.jpg
line_num: 11575; img_num: 8364; box_num: 1; 93_10.jpg
line_num: 11576; img_num: 8365; box_num: 1; 93_180.jpg
line_num: 11577; img_num: 8366; box_num: 1; 93_181.jpg
line_num: 11578; img_num: 8367; box_num: 1; 93_182.jpg
line_num: 11579; img_num: 8368; box_num: 1; 93_183.jpg
line_num: 11580; img_num: 8369; box_num: 1; 93_250.jpg
line_num: 11581; img_num: 8370; box_num: 1; 93_251.jpg
line_num: 11582; img_num: 8371; box_num: 1; 93_253.jpg
line_num: 11583; img_num: 8372; box_num: 1; 93_254.jpg
line_num: 11584; img_num: 8

8. Convert verification data to YOLO format

In [10]:
import os, sys
import glob
import random
from PIL import Image
import json

# Create yolo txt
def Create_txt(yolo_path, yolo_name, img_path, img_name, all_loc, class_dict,label_list):
    yolo = os.path.join(yolo_path, yolo_name)
    if os.path.exists(yolo):
        os.remove(yolo)
     # write object info into txt
    with open(yolo, "w") as f:
        for index,loc in enumerate(all_loc):
            img = Image.open(os.path.join(img_path, img_name))
            [img_width, img_height] = img.size
        
            # Further check the data, some labeling information may have w or h being 0, 
            # such data will lead to the calculation of regression loss as nan
            xmin = min(max(0, all_loc[index][0]), img_width)
            ymin = min(max(0, all_loc[index][1]), img_height)
            xmax = min(max(0, all_loc[index][2]), img_width)
            ymax = min(max(0, all_loc[index][3]), img_height)
            if xmax <= xmin or ymax <= ymin:
                print("Warning: in '{}' xml, there are some bbox w/h <=0")
                continue
    
            xcenter = xmin + (xmax - xmin) / 2
            ycenter = ymin + (ymax - ymin) / 2
            w = xmax - xmin
            h = ymax - ymin

      
            xcenter = round(xcenter / img_width, 6)
            ycenter = round(ycenter / img_height, 6)
            w = round(w / img_width, 6)
            h = round(h / img_height, 6)
            class_index = class_dict[label_list[index]]
            info = [str(i) for i in [class_index, xcenter, ycenter, w, h]]
            if index == 0:
                f.write(" ".join(info))
            else:
                f.write("\n" + " ".join(info))
file = 'data/fish_val.txt'
img_dir = "data//fish_test"
txt_dir = 'data/val_txt'
label_json_path = 'data/fish_classes.json'
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

img_num = 0
line_num = 0
img_list = []
# read class_indict
json_file = open(label_json_path, 'r')
class_dict = json.load(json_file)
with open(file, 'r') as fin:
    lines = fin.readlines()
    for idx in range(len(lines)):
        label_list = []
        all_loc = []
        line_num += 1

        img_name, label, label1, xmin, ymin, xmax, ymax = lines[idx].strip().split(' ')
        #img_name='test_' + img_name
        if img_name not in img_list:
            img_list.append(img_name)
            img_num += 1
            
            label_list.append(label + ' ' + label1)
            all_loc.append([int(xmin), int(ymin), int(xmax), int(ymax)])

            for idx2 in range(idx+1, len(lines)):
                img_name2, label2, label3, xmin2, ymin2, xmax2, ymax2 = lines[idx2].strip().split(' ')
                if img_name == img_name2:
                    label_list.append(label2 + ' ' + label3)
                    all_loc.append([int(xmin2), int(ymin2), int(xmax2), int(ymax2)])

            anno_name = str(img_name.split('.')[0]) + '.txt'
            Create_txt(txt_dir, anno_name, img_dir, img_name, all_loc,class_dict,label_list)

        print ('line_num: ' + str(line_num) + '; ' + 'img_num: ' + str(img_num) + '; '  + 'box_num: ' + str(len(all_loc)) + '; ' + img_name)

line_num: 1; img_num: 1; box_num: 1; 21_0.jpg
line_num: 2; img_num: 2; box_num: 1; 21_1.jpg
line_num: 3; img_num: 3; box_num: 1; 21_19.jpg
line_num: 4; img_num: 4; box_num: 1; 21_21.jpg
line_num: 5; img_num: 5; box_num: 1; 21_31.jpg
line_num: 6; img_num: 6; box_num: 1; 21_38.jpg
line_num: 7; img_num: 7; box_num: 1; 21_42.jpg
line_num: 8; img_num: 8; box_num: 1; 21_45.jpg
line_num: 9; img_num: 9; box_num: 1; 21_49.jpg
line_num: 10; img_num: 10; box_num: 1; 21_52.jpg
line_num: 11; img_num: 11; box_num: 1; 21_54.jpg
line_num: 12; img_num: 12; box_num: 1; 21_59.jpg
line_num: 13; img_num: 13; box_num: 1; 21_60.jpg
line_num: 14; img_num: 14; box_num: 2; 21_66.jpg
line_num: 15; img_num: 14; box_num: 0; 21_66.jpg
line_num: 16; img_num: 15; box_num: 1; 21_67.jpg
line_num: 17; img_num: 16; box_num: 2; 21_70.jpg
line_num: 18; img_num: 16; box_num: 0; 21_70.jpg
line_num: 19; img_num: 17; box_num: 1; 21_73.jpg
line_num: 20; img_num: 18; box_num: 1; 21_76.jpg
line_num: 21; img_num: 19; box_num: 2; 2

line_num: 189; img_num: 181; box_num: 1; 29_220.jpg
line_num: 190; img_num: 182; box_num: 1; 29_223.jpg
line_num: 191; img_num: 183; box_num: 1; 29_224.jpg
line_num: 192; img_num: 184; box_num: 1; 29_225.jpg
line_num: 193; img_num: 185; box_num: 1; 29_231.jpg
line_num: 194; img_num: 186; box_num: 1; 29_233.jpg
line_num: 195; img_num: 187; box_num: 1; 29_234.jpg
line_num: 196; img_num: 188; box_num: 1; 29_235.jpg
line_num: 197; img_num: 189; box_num: 1; 29_236.jpg
line_num: 198; img_num: 190; box_num: 1; 29_237.jpg
line_num: 199; img_num: 191; box_num: 1; 29_239.jpg
line_num: 200; img_num: 192; box_num: 1; 29_240.jpg
line_num: 201; img_num: 193; box_num: 1; 29_241.jpg
line_num: 202; img_num: 194; box_num: 1; 29_293.jpg
line_num: 203; img_num: 195; box_num: 1; 29_294.jpg
line_num: 204; img_num: 196; box_num: 1; 29_296.jpg
line_num: 205; img_num: 197; box_num: 1; 30_1.jpg
line_num: 206; img_num: 198; box_num: 1; 30_6.jpg
line_num: 207; img_num: 199; box_num: 1; 30_7.jpg
line_num: 208; img

line_num: 382; img_num: 367; box_num: 1; 35_31.jpg
line_num: 383; img_num: 368; box_num: 1; 35_34.jpg
line_num: 384; img_num: 369; box_num: 1; 35_44.jpg
line_num: 385; img_num: 370; box_num: 1; 35_59.jpg
line_num: 386; img_num: 371; box_num: 1; 35_60.jpg
line_num: 387; img_num: 372; box_num: 1; 35_63.jpg
line_num: 388; img_num: 373; box_num: 1; 35_64.jpg
line_num: 389; img_num: 374; box_num: 1; 35_67.jpg
line_num: 390; img_num: 375; box_num: 1; 35_75.jpg
line_num: 391; img_num: 376; box_num: 1; 35_77.jpg
line_num: 392; img_num: 377; box_num: 1; 35_79.jpg
line_num: 393; img_num: 378; box_num: 2; 35_83.jpg
line_num: 394; img_num: 378; box_num: 0; 35_83.jpg
line_num: 395; img_num: 379; box_num: 1; 35_86.jpg
line_num: 396; img_num: 380; box_num: 1; 35_88.jpg
line_num: 397; img_num: 381; box_num: 1; 35_117.jpg
line_num: 398; img_num: 382; box_num: 2; 35_124.jpg
line_num: 399; img_num: 382; box_num: 0; 35_124.jpg
line_num: 400; img_num: 383; box_num: 2; 35_140.jpg
line_num: 401; img_num: 383

line_num: 597; img_num: 570; box_num: 1; 40_124.jpg
line_num: 598; img_num: 571; box_num: 1; 40_127.jpg
line_num: 599; img_num: 572; box_num: 1; 40_147.jpg
line_num: 600; img_num: 573; box_num: 1; 40_163.jpg
line_num: 601; img_num: 574; box_num: 1; 40_166.jpg
line_num: 602; img_num: 575; box_num: 1; 40_169.jpg
line_num: 603; img_num: 576; box_num: 1; 40_171.jpg
line_num: 604; img_num: 577; box_num: 1; 40_176.jpg
line_num: 605; img_num: 578; box_num: 1; 40_184.jpg
line_num: 606; img_num: 579; box_num: 1; 40_245.jpg
line_num: 607; img_num: 580; box_num: 1; 40_252.jpg
line_num: 608; img_num: 581; box_num: 1; 40_256.jpg
line_num: 609; img_num: 582; box_num: 1; 41_4.jpg
line_num: 610; img_num: 583; box_num: 1; 41_8.jpg
line_num: 611; img_num: 584; box_num: 1; 41_11.jpg
line_num: 612; img_num: 585; box_num: 1; 41_17.jpg
line_num: 613; img_num: 586; box_num: 1; 41_24.jpg
line_num: 614; img_num: 587; box_num: 1; 41_200.jpg
line_num: 615; img_num: 588; box_num: 1; 41_205.jpg
line_num: 616; img_

line_num: 830; img_num: 790; box_num: 1; 49_224.jpg
line_num: 831; img_num: 791; box_num: 1; 49_226.jpg
line_num: 832; img_num: 792; box_num: 1; 49_234.jpg
line_num: 833; img_num: 793; box_num: 1; 49_238.jpg
line_num: 834; img_num: 794; box_num: 1; 49_243.jpg
line_num: 835; img_num: 795; box_num: 1; 49_244.jpg
line_num: 836; img_num: 796; box_num: 1; 49_253.jpg
line_num: 837; img_num: 797; box_num: 1; 49_254.jpg
line_num: 838; img_num: 798; box_num: 1; 49_258.jpg
line_num: 839; img_num: 799; box_num: 1; 49_267.jpg
line_num: 840; img_num: 800; box_num: 1; 49_269.jpg
line_num: 841; img_num: 801; box_num: 1; 49_273.jpg
line_num: 842; img_num: 802; box_num: 1; 50_18.jpg
line_num: 843; img_num: 803; box_num: 1; 50_19.jpg
line_num: 844; img_num: 804; box_num: 1; 50_21.jpg
line_num: 845; img_num: 805; box_num: 1; 50_26.jpg
line_num: 846; img_num: 806; box_num: 1; 50_27.jpg
line_num: 847; img_num: 807; box_num: 1; 50_31.jpg
line_num: 848; img_num: 808; box_num: 1; 50_33.jpg
line_num: 849; img_

line_num: 1111; img_num: 1040; box_num: 2; 53_281.jpg
line_num: 1112; img_num: 1040; box_num: 0; 53_281.jpg
line_num: 1113; img_num: 1041; box_num: 1; 53_284.jpg
line_num: 1114; img_num: 1042; box_num: 1; 53_285.jpg
line_num: 1115; img_num: 1043; box_num: 1; 53_286.jpg
line_num: 1116; img_num: 1044; box_num: 1; 53_288.jpg
line_num: 1117; img_num: 1045; box_num: 1; 53_289.jpg
line_num: 1118; img_num: 1046; box_num: 1; 53_290.jpg
line_num: 1119; img_num: 1047; box_num: 2; 53_291.jpg
line_num: 1120; img_num: 1047; box_num: 0; 53_291.jpg
line_num: 1121; img_num: 1048; box_num: 1; 53_292.jpg
line_num: 1122; img_num: 1049; box_num: 1; 53_296.jpg
line_num: 1123; img_num: 1050; box_num: 1; 53_297.jpg
line_num: 1124; img_num: 1051; box_num: 1; 54_8.jpg
line_num: 1125; img_num: 1052; box_num: 1; 54_22.jpg
line_num: 1126; img_num: 1053; box_num: 1; 54_25.jpg
line_num: 1127; img_num: 1054; box_num: 1; 54_27.jpg
line_num: 1128; img_num: 1055; box_num: 1; 54_29.jpg
line_num: 1129; img_num: 1056; box

line_num: 1435; img_num: 1341; box_num: 1; 67_32.jpg
line_num: 1436; img_num: 1342; box_num: 1; 67_34.jpg
line_num: 1437; img_num: 1343; box_num: 1; 67_38.jpg
line_num: 1438; img_num: 1344; box_num: 1; 67_52.jpg
line_num: 1439; img_num: 1345; box_num: 1; 67_158.jpg
line_num: 1440; img_num: 1346; box_num: 1; 67_161.jpg
line_num: 1441; img_num: 1347; box_num: 2; 68_12.jpg
line_num: 1442; img_num: 1347; box_num: 0; 68_12.jpg
line_num: 1443; img_num: 1348; box_num: 1; 68_15.jpg
line_num: 1444; img_num: 1349; box_num: 1; 68_16.jpg
line_num: 1445; img_num: 1350; box_num: 1; 68_31.jpg
line_num: 1446; img_num: 1351; box_num: 1; 68_35.jpg
line_num: 1447; img_num: 1352; box_num: 1; 68_36.jpg
line_num: 1448; img_num: 1353; box_num: 1; 68_37.jpg
line_num: 1449; img_num: 1354; box_num: 1; 68_47.jpg
line_num: 1450; img_num: 1355; box_num: 1; 68_49.jpg
line_num: 1451; img_num: 1356; box_num: 1; 68_52.jpg
line_num: 1452; img_num: 1357; box_num: 1; 68_60.jpg
line_num: 1453; img_num: 1358; box_num: 1; 6

line_num: 1628; img_num: 1527; box_num: 1; 78_1.jpg
line_num: 1629; img_num: 1528; box_num: 2; 78_2.jpg
line_num: 1630; img_num: 1528; box_num: 0; 78_2.jpg
line_num: 1631; img_num: 1529; box_num: 1; 78_3.jpg
line_num: 1632; img_num: 1530; box_num: 1; 78_4.jpg
line_num: 1633; img_num: 1531; box_num: 1; 78_6.jpg
line_num: 1634; img_num: 1532; box_num: 1; 78_10.jpg
line_num: 1635; img_num: 1533; box_num: 1; 78_12.jpg
line_num: 1636; img_num: 1534; box_num: 1; 78_14.jpg
line_num: 1637; img_num: 1535; box_num: 2; 78_15.jpg
line_num: 1638; img_num: 1535; box_num: 0; 78_15.jpg
line_num: 1639; img_num: 1536; box_num: 1; 78_20.jpg
line_num: 1640; img_num: 1537; box_num: 1; 78_21.jpg
line_num: 1641; img_num: 1538; box_num: 1; 78_23.jpg
line_num: 1642; img_num: 1539; box_num: 1; 78_27.jpg
line_num: 1643; img_num: 1540; box_num: 1; 78_30.jpg
line_num: 1644; img_num: 1541; box_num: 2; 78_31.jpg
line_num: 1645; img_num: 1541; box_num: 0; 78_31.jpg
line_num: 1646; img_num: 1542; box_num: 1; 78_34.jpg

line_num: 1866; img_num: 1749; box_num: 1; 85_266.jpg
line_num: 1867; img_num: 1750; box_num: 1; 85_279.jpg
line_num: 1868; img_num: 1751; box_num: 1; 85_284.jpg
line_num: 1869; img_num: 1752; box_num: 1; 85_290.jpg
line_num: 1870; img_num: 1753; box_num: 1; 85_298.jpg
line_num: 1871; img_num: 1754; box_num: 1; 86_4.jpg
line_num: 1872; img_num: 1755; box_num: 1; 86_67.jpg
line_num: 1873; img_num: 1756; box_num: 1; 86_72.jpg
line_num: 1874; img_num: 1757; box_num: 1; 86_130.jpg
line_num: 1875; img_num: 1758; box_num: 1; 86_141.jpg
line_num: 1876; img_num: 1759; box_num: 1; 86_142.jpg
line_num: 1877; img_num: 1760; box_num: 1; 86_152.jpg
line_num: 1878; img_num: 1761; box_num: 1; 87_2.jpg
line_num: 1879; img_num: 1762; box_num: 1; 87_16.jpg
line_num: 1880; img_num: 1763; box_num: 1; 87_17.jpg
line_num: 1881; img_num: 1764; box_num: 1; 87_18.jpg
line_num: 1882; img_num: 1765; box_num: 1; 87_20.jpg
line_num: 1883; img_num: 1766; box_num: 1; 87_118.jpg
line_num: 1884; img_num: 1767; box_num